In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    

Using TensorFlow backend.


In [13]:
datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    #rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# I need to identify the correct filepath and then I will reshape my images, train, and test

#is the filepath in the same folder or in a global directory?  My files are saved in users/Vanessa/Desktop/Data_Science/train/cats or /dog

##fixed!!!!!

In [14]:
img=load_img('train/cats/cat.0.jpg')
x = img_to_array(img)
x = x.reshape((1,)+ x.shape)
i=0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview',save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break


In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 150, 150

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [16]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [17]:

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [18]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


test_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
#model.fit_generator(
#    train_generator,
#    steps_per_epoch = nb_train_samples // batch_size, 
#    epochs = epochs, 
#    validation_data = nb_validation_samples // batch_size,
#    validation_steps = 50)
#mode.save_weights('first_try.h5')

In [ ]:
generator = datagen.flow_from_directory(
    'train',
    target_size=(150,150),
    batch_size = batch_size,
    class_mode=None,
    shuffle=False)

In [41]:
import numpy as np
bottleneck_features_train = model.predict_generator(generator, 2000)
np.save('bottleneck_features_train.npy', bottleneck_features_train)

In [42]:
validation_generator = datagen.flow_from_directory(
    'validation',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 800 images belonging to 2 classes.


In [44]:
bottleneck_features_validation = model.predict_generator(validation_generator, 800)
np.save('bottleneck_featuers_validation.npy', bottleneck_features_validation)

In [46]:
train_data = np.load('bottleneck_features_train.npy')
train_labels = np.array([0] * 1000 + [1] * 1000)

In [49]:
validation_data = np.load('bottleneck_featuers_validation.npy')
validation_labels = np.array([0]*400 + [1] * 400)


In [53]:
#print(validation_data)

In [66]:
model = Sequential()
model.add(Flatten(input_shape = train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer flatten_8: expected min_ndim=3, found ndim=2

In [60]:
model.fit(train_data, train_labels,
        epochs=50,
        batch_size=batch_size,
         validation_data = (validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

RuntimeError: The model needs to be compiled before being used.